<a href="https://colab.research.google.com/github/rdivi297/DataPreprocessing_from_scratch/blob/main/DataPrerprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
import pandas as pd

In [10]:
df=pd.read_csv("/content/drive/MyDrive/weather.csv")
print(df.head())

    Outlook  Temp Humidity  Windy  Play
0     rainy   hot     high      0     0
1     rainy   hot     high      1     0
2  overcast   hot     high      0     1
3     sunny  mild     high      0     1
4     sunny  cool   normal      0     1


In [14]:
def find_entropy(df):
  label=df.keys()[-1]
  entropy=0
  values=df[label].unique()
  for value in values:
    prob=len(df[df[label]==value])/len(df)
    entropy-=prob*np.log2(prob)
  return entropy
find_entropy(df)

0.9402859586706311

In [22]:
def avg_info_entropy(df,attribute):
  target=df.keys()[-1]
  target_values=df[target].unique()
  attribute_values=df[attribute].unique()
  avgentropy=0
  for value in attribute_values:
    entropy=0
    for value1 in target_values:
      num=len(df[attribute][df[attribute]==value][df[target]==value1])
      den=len(df[attribute][df[attribute]==value])
      prob=num/den
      entropy-=prob*np.log2(prob+1e-7)
    weight=den/len(df)
    avgentropy+=entropy*weight
  return avgentropy
avg_info_entropy(df,'Windy')



0.8921586397233878

In [23]:
def winner(df):
  attributes=df.keys()[:-1]
  IG=[]
  for attribute in attributes:
    IG.append(find_entropy(df)-avg_info_entropy(df,attribute))
  print(IG)
  return attributes[np.argmax(IG)]
winner(df)

[0.24675006709356562, 0.029222854197929804, 0.15183578990130564, 0.048127318947243314]


'Outlook'

In [32]:
def training(df,tree=None):
  node=winner(df)
  if tree is None:
    tree={}
    tree[node]={}
  node_unique_values=df[node].unique()
  target=df.keys()[-1]
  for value in node_unique_values:
    sub_df=df[df[node]==value]
    target_unique=sub_df[target].unique()
    if(len(target_unique)==1):
      tree[node][value]=target_unique[0]
    else:
      tree[node][value]=training(sub_df)
  return tree
tree=training(df)




[0.24675006709356562, 0.029222854197929804, 0.15183578990130564, 0.048127318947243314]
[2.8853897804381745e-07, 0.5709507964319583, 0.9709507387241655, 0.019973382560951936]
[2.8853897804381745e-07, 0.019973382560951936, 0.019973382560951936, 0.9709507387241655]


In [39]:
def prediction(instance,tree):
  for node in tree.keys():
    value=instance[node]
    subtree=tree[node][value]
    label=0
    if type(subtree) is dict:
      label=prediction(instance,subtree)
    else:
      label=subtree
      break;
  return label


In [40]:
df2=pd.read_csv("/content/drive/MyDrive/weather_test.csv")
y_label=[]
for i in range(len(df2)):
  y_label.append(prediction(df2.iloc[i,:],tree))
y_label

[0, 1]

In [42]:
from sklearn.metrics import classification_report
print(classification_report(df2.iloc[:,-1],y_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

